In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
spec_data = [
#      (pd.read_csv("fixed_flags/fix_flags.bc.20180813-135517.false.csv"), 'false'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180812-142333.spread.csv"), 'spread'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180813-025019.close.csv"), 'close'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180814-034312.true.csv"), 'true')
]
npb_data = [
#      (pd.read_csv("fixed_flags/fix_flags.bc.20180813-124212.false.csv"), 'false'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180812-142221.spread.csv"), 'spread'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180812-204259.close.csv"), 'close'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180814-034012.true.csv"), 'true')
]

parboil_data = [
#      (pd.read_csv("fixed_flags/fix_flags.bc.20180826-200104.false.csv"), 'false'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180826-232755.spread.csv"), 'spread'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180826-220610.close.csv"), 'close'),
    (pd.read_csv("fixed_flags/fix_flags.bc.20180826-180300.true.csv"), 'true')
]

rodinia_data = [
    (pd.read_csv("fixed_flags/fix_flags.bc.20180830-034651.true.csv"), 'true')
]

for frame in parboil_data + npb_data + spec_data + rodinia_data:
    frame[0].loc[:, 'Energy'] *= 1e-6 # Convert energy to Joules
    frame[0].loc[:, 'Benchmark'] = frame[0]['Benchmark'].str.replace('.x', '')
    
    

In [ ]:
print(spec_data[0][0]["Benchmark"].unique())
print(npb_data[0][0]["Benchmark"].unique())
print(parboil_data[0][0]["Benchmark"].unique())
print(rodinia_data[0][0]["Benchmark"].unique())

In [ ]:
flags = ['O1', 'O2', 'O3']
markers = ['x', 'o', 'v']

def compare_results(frames, benchmarks=None):
    if benchmarks == None:
        benchmarks = frames[0][0]["Benchmark"].unique()
    
    rows = len(benchmarks)
    columns = len(frames)

    fig = plt.figure(figsize=(8.0*columns, 6.0*rows))
    
    benchmark_axes = {}
    
    for i, (frame, label) in enumerate(frames):
        plot_pos = i + 1
        
#         for benchmark in benchmarks:
        for benchmark in benchmarks:
            benchmark_data = frame[frame["Benchmark"] == benchmark]
            
            if benchmark in benchmark_axes:
                ax = fig.add_subplot(rows, columns, plot_pos, sharex=benchmark_axes[benchmark], sharey=benchmark_axes[benchmark])
            else:
                ax = fig.add_subplot(rows, columns, plot_pos)
                benchmark_axes[benchmark] = ax
                
            plot_pos += columns
            
            for k, flag in enumerate(flags):

                marker = markers[k]

                flag_data = benchmark_data[benchmark_data["Flags"] == "-" + flag]

                x = flag_data["Time"].values
                y = flag_data["Energy"].values

                ax.scatter(x, y, marker=marker, label=flag)
            
            ax.legend()
            ax.set_title(benchmark + ', proc_bind=' + label)
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Energy (J)')
            ax.grid()

def plot_single_benchmark(frames, benchmark, proc_bind):
    frame = None
    for f in frames:
        if f[1] == proc_bind:
            frame = f[0]
            
    benchmark_data = frame[frame["Benchmark"].str.lower() == benchmark]
    
    plt.figure()
    for k, flag in enumerate(flags):
        marker = markers[k]

        flag_data = benchmark_data[benchmark_data["Flags"] == "-" + flag]

        x = flag_data["Time"].values
        y = flag_data["Energy"].values

        plt.scatter(x, y, marker=marker, label=flag)
        
    plt.legend()
    plt.title('Energy vs Time for standard optimisation levels for ' + benchmark)
    plt.xlabel('Time (s)')
    plt.ylabel('Energy (J)')
    plt.grid()
    plt.show()
    

In [ ]:
# compare_results(npb_data)
# compare_results(spec_data)
# compare_results(parboil_data)
# compare_results(rodinia_data)

In [ ]:
def average_energy_time(frames, proc_bind):
    frame = None
    for f in frames:
        if f[1] == proc_bind:
            frame = f[0]
            
    average_energy = np.mean(frame['Energy'])
    average_time = np.mean(frame['Time'])
    
    return average_energy, average_time

In [ ]:
print(average_energy_time(npb_data, 'close'))
print(average_energy_time(npb_data, 'spread'))
print(average_energy_time(npb_data, 'true'))

In [ ]:
print(average_energy_time(spec_data, 'close'))
print(average_energy_time(spec_data, 'spread'))
print(average_energy_time(spec_data, 'true'))

In [ ]:
plot_single_benchmark(npb_data, 'bt.c', 'true')
plot_single_benchmark(npb_data, 'is.d', 'true')

In [ ]:
def summary(data, variable):
    mean = np.mean(data[variable].values)
    median = np.median(data[variable].values)
    min = np.min(data[variable].values)
    max = np.max(data[variable].values)
    range = max - min
    relative_range = range / mean * 100
    std = np.std(data[variable].values)
    coeff_var = std / mean * 100
    
    return [mean, median, min, max, range, relative_range, std, coeff_var]

def summary_frame(array):
    return pd.DataFrame(array[:,1:], 
                       index=array[:,0],
                       dtype=float,
                       columns=['Mean', 'Median', 'Min', 'Max', 'Range', 'Relative Range', 'Std', 'CV'])

In [ ]:
spec_true_data = spec_data[-1][0]
npb_true_data = npb_data[-1][0]
parboil_true_data = parboil_data[-1][0]
rodinia_true_data = rodinia_data[-1][0]
true_data = pd.concat([spec_true_data, npb_true_data, parboil_true_data, rodinia_true_data])

summary_list_time = []
summary_list_energy = []

for benchmark in true_data["Benchmark"].unique():
    benchmark_data = true_data[(true_data["Benchmark"] == benchmark) & (true_data["Flags"] == '-O3')]
    
    summary_list_time.append([benchmark] + summary(benchmark_data, 'Time'))
    summary_list_energy.append([benchmark] + summary(benchmark_data, 'Energy'))

summary_frame_time = summary_frame(np.array(summary_list_time))
summary_frame_energy = summary_frame(np.array(summary_list_energy))

In [ ]:
summary_frame_time

In [ ]:
def stable_benchmarks(metric, threshold_percentage=2):
    low_variation_time = summary_frame_time[summary_frame_time[metric] <= threshold_percentage].index
    low_variation_energy = summary_frame_energy[summary_frame_energy[metric] <= threshold_percentage].index
    return set(low_variation_time).intersection(set(low_variation_energy))

In [ ]:
benchmarks_for_ce = stable_benchmarks('CV')
print(len(benchmarks_for_ce))
print(benchmarks_for_ce)